In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from functools import reduce
from matplotlib import pyplot as plt
import seaborn as sns

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# set of external covariates
population = pd.read_csv('datasets/popolazione2019.csv') 

population['SchoolClo'] = population['SchoolClo'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime('%Y-%m-%d'))
population['Lockdown'] = population['Lockdown'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime('%Y-%m-%d'))
population['Manufactur'] = population['Manufactur'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime('%Y-%m-%d'))

In [3]:
# load updated regional covid data from json
# Source: Dipartimento della Protezione Civile

url = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json"
json_data = pd.read_json(url)
json_data.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None


In [4]:
n_json = len(json_data)
covid = {
    'date': json_data.data,
    'region': json_data.denominazione_regione,
    'icu': json_data.terapia_intensiva,
    'positives': json_data.totale_positivi,
    'deaths': json_data.deceduti,
    'cases': json_data.totale_casi,
    'swabs': json_data.tamponi,
}

# Remove time from dates
FMT = '%Y-%m-%dT%H:%M:%S' #datetime format
covid['date'] = covid['date'].apply(lambda x: datetime.strptime(x, FMT).strftime('%Y-%m-%d'))

In [5]:
datetoday = covid['date'].max()
print("Most recent date is", datetoday)

dates = covid['date'].unique()

Most recent date is 2021-06-24


In [6]:
# define time variable
x = np.arange(0, len(dates))
m = len(x)

# look at regions in covid data
regions = covid['region'].unique()
n_reg  = len(regions)

# create matrices with CUMULATIVE counts of deaths, cases and swabs
# and current counts for ICU and positives
covid['positives']

0           0
1           0
2           0
3           0
4          18
         ... 
10222    4753
10223    2555
10224     836
10225      35
10226    4827
Name: totale_positivi, Length: 10227, dtype: int64

In [7]:
DEAcum = pd.DataFrame(0, index=regions, columns=dates, dtype='int')
CAScum = pd.DataFrame(0, index=regions, columns=dates, dtype='int')
SWABcum = pd.DataFrame(0, index=regions, columns=dates, dtype='int')
ICUt = pd.DataFrame(0, index=regions, columns=dates, dtype='int')
POSt = pd.DataFrame(0, index=regions, columns=dates, dtype='int')

for i in range(0, n_reg):
    deaths = covid['deaths'].loc[json_data['denominazione_regione'] == regions[i]]
    cases = covid['cases'].loc[json_data['denominazione_regione'] == regions[i]]
    swabs = covid['swabs'].loc[json_data['denominazione_regione'] == regions[i]]
    icut = covid['icu'].loc[json_data['denominazione_regione'] == regions[i]]
    post = covid['positives'].loc[json_data['denominazione_regione'] == regions[i]]
    
    DEAcum.loc[regions[i]] = deaths.values
    CAScum.loc[regions[i]] = cases.values
    SWABcum.loc[regions[i]] = swabs.values
    ICUt.loc[regions[i]] = icut.values
    POSt.loc[regions[i]] = post.values


In [8]:
## wide format
regions_series = pd.Series(regions, index=regions, name='Territorio')

DEAcum_wide = pd.concat([regions_series, DEAcum], axis=1)
CAScum_wide = pd.concat([regions_series, CAScum], axis=1)
SWABcum_wide = pd.concat([regions_series, SWABcum], axis=1)
ICUt_wide = pd.concat([regions_series, ICUt], axis=1)
POSt_wide = pd.concat([regions_series, POSt], axis=1)

In [9]:
# long format
def pivot_longer(df, id_vars, value_vars, value_name, rename_col, sort_by):
    return pd.melt(df, id_vars=id_vars, value_vars=value_vars, value_name=value_name)\
                        .rename(columns=rename_col)\
                        .sort_values(by=sort_by)\
                        .reset_index(drop=True)

In [10]:
DEAcum_long = pivot_longer(DEAcum_wide, id_vars='Territorio', value_vars=dates, value_name='deathsCUM', rename_col={'variable': 'Date'}, sort_by=['Territorio', 'Date'])
CAScum_long = pivot_longer(CAScum_wide, id_vars='Territorio', value_vars=dates, value_name='casesCUM', rename_col={'variable': 'Date'}, sort_by=['Territorio', 'Date'])
SWABcum_long = pivot_longer(SWABcum_wide, id_vars='Territorio', value_vars=dates, value_name='swabsCUM', rename_col={'variable': 'Date'}, sort_by=['Territorio', 'Date'])
ICUt_long = pivot_longer(ICUt_wide, id_vars='Territorio', value_vars=dates, value_name='icu', rename_col={'variable': 'Date'}, sort_by=['Territorio', 'Date'])
POSt_long = pivot_longer(POSt_wide, id_vars='Territorio', value_vars=dates, value_name='positives', rename_col={'variable': 'Date'}, sort_by=['Territorio', 'Date'])

In [11]:
# we need daily number of counts, deaths and swabs
DEAt_long = DEAcum_long.copy()
CASt_long = CAScum_long.copy()
SWABt_long = SWABcum_long.copy()

DEAt_long['deaths'] = DEAcum_long['deathsCUM'] - DEAcum_long.groupby(['Territorio'])['deathsCUM'].shift(1)
CASt_long['cases'] = CAScum_long['casesCUM'] - CAScum_long.groupby(['Territorio'])['casesCUM'].shift(1)
SWABt_long['swabs'] = SWABcum_long['swabsCUM'] - SWABcum_long.groupby(['Territorio'])['swabsCUM'].shift(1)

In [12]:
# expand dataset
t = np.repeat(population['Territorio'], len(dates))
d = pd.Series(np.tile(dates, n_reg))

my_data = pd.DataFrame({'Territorio': t.values, 'Date': d.values })

# attach time-unvarying covariates
my_data2 = pd.merge(my_data, population, how="left", on=["Territorio"])
my_data2

,Territorio,Date,Pop2019,ICU2019,Pop65,SchoolClo,Lockdown,Manufactur,GDP,GDPcap,Density2019,RSA,RSAp,DepRatio,DepRatioOld,House2019
0,Valle d'Aosta,2020-02-24,125666,10,23.5,2020-03-04,2020-03-09,2020-03-22,4902.0,39008.164500,38.537236,2,0.083368,58.7,37.8,2.0
1,Valle d'Aosta,2020-02-25,125666,10,23.5,2020-03-04,2020-03-09,2020-03-22,4902.0,39008.164500,38.537236,2,0.083368,58.7,37.8,2.0
2,Valle d'Aosta,2020-02-26,125666,10,23.5,2020-03-04,2020-03-09,2020-03-22,4902.0,39008.164500,38.537236,2,0.083368,58.7,37.8,2.0
3,Valle d'Aosta,2020-02-27,125666,10,23.5,2020-03-04,2020-03-09,2020-03-22,4902.0,39008.164500,38.537236,2,0.083368,58.7,37.8,2.0
4,Valle d'Aosta,2020-02-28,125666,10,23.5,2020-03-04,2020-03-09,2020-03-22,4902.0,39008.164500,38.537236,2,0.083368,58.7,37.8,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10222,Sardegna,2021-06-20,1639591,120,23.2,2020-03-04,2020-03-09,2020-03-22,34541.7,21067.266166,68.032763,16,0.666945,53.8,36.5,2.2
10223,Sardegna,2021-06-21,1639591,120,23.2,2020-03-04,2020-03-09,2020-03-22,34541.7,21067.266166,68.032763,16,0.666945,53.8,36.5,2.2
10224,Sardegna,2021-06-22,1639591,120,23.2,2020-03-04,2020-03-09,2020-03-22,34541.7,21067.266166,68.032763,16,0.666945,53.8,36.5,2.2
10225,Sardegna,2021-06-23,1639591,120,23.2,2020-03-04,2020-03-09,2020-03-22,34541.7,21067.266166,68.032763,16,0.666945,53.8,36.5,2.2


In [13]:
# create dummies for time-varing covariates
# (school closure, start lockdown)
my_data2['SchoolD'] = np.where(my_data2['SchoolClo'] <= my_data2['Date'], 1, 0)
my_data2['LockD'] = np.where(my_data2['Lockdown'] <= my_data2['Date'], 1, 0)
my_data2['ManufactD'] = np.where(my_data2['Manufactur'] <= my_data2['Date'], 1, 0)

In [14]:
# attach COVID cases, deaths, swabs and current ICU use
data_frames_to_merge = [my_data2, DEAt_long, CASt_long, SWABt_long, ICUt_long, POSt_long]
my_data3 = reduce(lambda left, right: pd.merge(left, right, on=["Territorio", "Date"]), data_frames_to_merge)
my_data3['deaths'] = np.where(my_data3['deaths'] < 0, 0, my_data3['deaths'])

In [15]:
# define start of epidemic as day when cumulative cases
# surpass total population / 10^6
population['thresh_epi'] = np.round(population['Pop2019'] * 1e-6, 0).astype('int')
population['thresh_epi'] = np.where(population['thresh_epi'] == 0, 1, population['thresh_epi'])

df_startepi = population[['Territorio', 'thresh_epi']]
# attach start epidemic and delay

In [16]:
my_data4 = pd.merge(my_data3, df_startepi, how="left", on=["Territorio"])

def mutate1(data):
    data['startEpiD'] = np.where(data.casesCUM >= data.thresh_epi, 1, 0)
    data['startEpi'] = data['Date'][data['startEpiD'] == 1].min()

    return data

my_data5 = my_data4.groupby(['Territorio']).apply(mutate1)
my_data5['startEpi'] = pd.to_datetime(my_data5['startEpi'])
my_data5['Date'] = pd.to_datetime(my_data5['Date'])
my_data5['DelayEpi'] = (my_data5['startEpi'] - my_data5['Date'].min()).dt.days

In [17]:
df_covid = my_data5.copy().dropna()
# transform variables
df_covid['swabsCUM'] = df_covid['swabsCUM'] / 1000
df_covid['time'] = np.tile(np.arange(1, len(dates)), n_reg)
df_covid['lGDPcap'] = np.round(np.log(df_covid.GDPcap), 2)

def mutate2(data):
    data['icupch'] = np.round((data.icu / data.ICU2019) * 100, 2)
    data['sqrtICU'] = np.round(np.sqrt(data.icupch), 2)
    data['sqrtPOS'] = np.round(np.sqrt(data.positives), 2)

    return data

df_covid = df_covid.groupby(['Territorio']).apply(mutate2)

In [18]:
df_covid['cases']

1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
         ... 
10222     9.0
10223     4.0
10224     6.0
10225    18.0
10226    16.0
Name: cases, Length: 10206, dtype: float64

In [19]:
# keep variables of interest and rename
columns_rename = {
    "Territorio": "region", 
    "Date": "date",
    "Pop2019": "population",
    "Pop65": "pop65",
    "Density2019": "density",
    "RSA": "nursehome",
    "swabsCUM": "swabs",
    "DelayEpi": "delayepi",
    "lGDPcap": "gdpcap",
    "sqrtICU": "icupch",
    "sqrtPOS": "positives",
    "House2019": "households"
}

var_of_interest = ['Territorio', 'Date', 'Pop2019', 'Pop65', 'Density2019', 'RSA','deaths','cases', 'swabsCUM', 'DelayEpi', 'time', 'lGDPcap','icu', 'sqrtICU','sqrtPOS', 'House2019']
df_covid = df_covid[var_of_interest].rename(columns=columns_rename)

In [20]:
df_covid.to_csv("datasets/dataCOVID_all.csv", index=False)

In [21]:
df_covid

,region,date,population,pop65,density,nursehome,deaths,cases,swabs,delayepi,time,gdpcap,icu,icupch,positives,households
1,Valle d'Aosta,2020-02-25,125666,23.5,38.537236,2,0.0,0.0,0.007,10,1,10.57,0,0.00,0.00,2.0
2,Valle d'Aosta,2020-02-26,125666,23.5,38.537236,2,0.0,0.0,0.007,10,2,10.57,0,0.00,0.00,2.0
3,Valle d'Aosta,2020-02-27,125666,23.5,38.537236,2,0.0,0.0,0.009,10,3,10.57,0,0.00,0.00,2.0
4,Valle d'Aosta,2020-02-28,125666,23.5,38.537236,2,0.0,0.0,0.009,10,4,10.57,0,0.00,0.00,2.0
5,Valle d'Aosta,2020-02-29,125666,23.5,38.537236,2,0.0,0.0,0.009,10,5,10.57,0,0.00,0.00,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10222,Sardegna,2021-06-20,1639591,23.2,68.032763,16,0.0,9.0,1354.772,9,482,9.96,5,2.04,107.60,2.2
10223,Sardegna,2021-06-21,1639591,23.2,68.032763,16,0.0,4.0,1355.601,9,483,9.96,5,2.04,49.92,2.2
10224,Sardegna,2021-06-22,1639591,23.2,68.032763,16,3.0,6.0,1360.214,9,484,9.96,4,1.82,49.41,2.2
10225,Sardegna,2021-06-23,1639591,23.2,68.032763,16,1.0,18.0,1364.083,9,485,9.96,4,1.82,49.05,2.2
